# Battle of Neighbourhoods - Week 1
This notebook consist of two parts.

1 Introduction/Business Problem

   1.a Discussion of the business problem and the audience who would be interested in this project.

2 Data Section

   2.a What data is used?
   2.b Importing Libraries
   2.c Credentials and Core location
   2.d Search for hotel & restaurant within 1 KM
   2.e Location of Hotels

### 1. Introduction/Business Problem
Discussion of the business problem and the audience who would be interested in this project.
Something about the tourist place - Dubai ,UAE
I am from Dubai,a beautiful tourist spot in Middle East.

Opening of Hotel/Restaurant Shop
Coming down to business problem,I would like to open a hotel/restaurant near golf Course.As it is a famous tourist spot,there is already lots of attention towards it.I know there will be many competitors in terms of hotel and restaurant.But keeping them in mind,i need to locate my hotel in place where more people are attracted and comfortable for a stay and a good meal.I want to bring foreign and local peoples attention towards my new hotel.I would like to flavour my restaurant recipe with Italian,American,typical south & north indian foods to grab their taste.
The challenge is to find a suitable location for opening a new hotel / restaurant attracted to all local and foreign people in the centre of all famous venues.

Expected / Interested Audience
40% local and 60% foreign peoples visit Dubai once in a year.Some people stay for couple of days or more.Also they find some place for hangout or a good meal.Their main focus might be belonging to stay somewhere near to reach venues.Apart from these set of people,students and working professionals are common audience here.So we may need to fascinate them all.




### 2.Data section
2.a What data is used?
We will be completely working on Foursquare data to explore and try to locate our new hotel where more venues like church, temples, beach, museums, memorials that are present nearby.

How will we be solving using this data?

We will looking for midpoint area of venues to locate our new hotel.Before that our major focus will be on all venues present in and around the core place of kanyakumari.

Just a heads up on how many hotels are distributed now around kanyakumari.We will perform some EDA on hotels & restaurants present in the tourist spot.On furthur notebook we will use Foursquare data to determine other venues as well.

In [1]:
# Import libraries
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("*********   Loaded library     ***********")

*********   Loaded library     ***********


In [4]:

CLIENT_ID = '2RRIH0BC5WFWWZBVFHXPSVDBBXQ4DUPPG2NH5KEC4F3DCMYA' # your Foursquare ID
CLIENT_SECRET = '0B05V0KYK04V0IU1CFLULTVPINOR1N2DAMLOFZWPOB0EJP3H' # your Foursquare Secret
VERSION = '20191028'
LIMIT = 150

address = "Dubai, UAE"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude   #8.079252 # location.latitude 
longitude =location.longitude #77.5499338 # location.longitude # 

Dub='Dubai location : {},{}'.format(latitude,longitude)
print(Dub)

Dubai location : 25.0750095,55.18876088183319


In [8]:
#Quering for hotel & restaurant

search_query = 'hotel'
search_query_res = 'restaurant'

radius = 2000
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

In [9]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()
#results_hotel

In [10]:

# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']
venues_restaurant = results_restaurant['response']['venues']

# tranform venues into a dataframe and merging both data
dataframe_hotel = json_normalize(venues_hotel)
dataframe_restaurant = json_normalize(venues_restaurant)

dataframe = pd.concat([dataframe_hotel,dataframe_restaurant])

print("There are {} restaurants and hotels in Dubai".format(dataframe.shape[0]))

There are 37 restaurants and hotels in Dubai


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [11]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

  
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered
hotels_df=dataframe_filtered[['name','categories','distance','lat','lng','id']]
hotels_df.head()

,name,categories,distance,lat,lng,id
0,Hotel Grand Belle Vue,Hotel,2562,25.094812,55.175802,4e9245825c5ce36b18f3724a
1,Somewhere Hotel Tecom,Hotel,2772,25.096047,55.174039,579985b1498ef275bc6682b5
2,Time Crystal Hotel Apartments,Residential Building (Apartment / Condo),2309,25.092794,55.176963,51cea368498e45cf2b028cef
3,MiINC Hotel Apartments,College Library,2451,25.094028,55.176503,4bd8c52d2ecdce7206f1d0f2
4,Layia Oak Hotel & Suites,Hotel,2907,25.096887,55.173009,4bd846256a51ce726bbcde3f


In [12]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around Dubai

# add a red circle marker to represent the core location of Dubai
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Kanyakumari',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(hotels_df.lat, hotels_df.lng, hotels_df.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map